1RSB
===

\begin{align*}
p\left(u\right)  &= \sum_{h_{1},\dots,h_{k}}\delta\left(u;\min_{j=1,\dots,k}\left|h_{j}\right|\prod_{j=1}^{k}\text{sign}\left(h_{j}\right)\right)\prod_{j=1}^{k}q_j\left(h_{j}\right)e^{y C_{ai}}\\
q(h) &= \sum_{u_1,\dots,u_d} \prod_{b=1,\dots,d} p_b(u_b) \delta\left(h-s-\sum_{b=1,\dots,d}u_b\right)e^{y C_{ia}}\\
\end{align*}

where
\begin{align*}
C_{ai} &= 0\\% -2\min_{j=1,\dots,k}|h_{j}|\Theta\left(-\prod_{j=1}^kh_{j}\right)\\
C_{ia} &=\left|s + \sum_{b=1}^d u_b\right| - \sum_{b=1}^d|u_b| = |h| - \sum_{b=1}^d|u_b|
\end{align*}



In [1]:
using OffsetArrays

const ∏ = prod
const ∑ = sum

sum (generic function with 14 methods)

In [2]:
function iter_slow_factor(Q, J, y=0.0)
    k = length(Q)
    p = fill(0.0, -J:J)
    for hs in Iterators.product(fill(-J:J,k)...)
        w = ∏(q[h] for (q,h) ∈ zip(Q,hs))
        u = minimum(abs.(hs))*sign(∏(hs))
        p[clamp(u, -J, J)] += w
    end
    p ./= sum(p)
end

function iter_slow_var(P, s, J, y=0.0) 
    q = fill(0.0, -J:J)
    for us in Iterators.product(fill(-J:J, length(P))...)
        h = sum(us) + s
        Cia = abs(h) - sum(abs.(us)) 
        w = ∏(p[u] for (p,u) ∈ zip(P,us)) * exp(y*Cia)
        q[clamp(h, -J, J)] += w 
    end
    q ./= sum(q)
end

iter_slow_var (generic function with 2 methods)

Simplifications
--

(not needed $C_{ai}=0$) -> {$C_{ai}$ can be simplified: 

\begin{align}
C_{ai} &= -2\min_{j\in\partial a\setminus i}|h_{ja}|\Theta\left(-u_{ai}\right)\\
&= -2|u_{ai}|\Theta(-u_{ai})\\
&= u_{ai}-|u_{ai}|
\end{align}

So}

\begin{align*}
p\left(u\right)= & \sum_{h_{1},\dots,h_{k}}\delta\left(u;\min_{j=1,\dots,k}\left|h_{j}\right|\prod_{j=1}^{k}\text{sign}\left(h_{j}\right)\right)\prod_{j=1}^{k}q_j\left(h_{j}\right)\\%e^{-y 2 u \Theta(-u)}\\
\end{align*}

To compute $p$, define

\begin{align*}
a_k(u) &= \mathbb{P}\left\{\left(\textrm{sign}\left(\prod_{i=1}^k h_i\right)=\textrm{sign}(u) \right)\wedge \left(|h_1|,\dots,|h_k| \ge |u|\right)\right\}\\
\end{align*}

$a_k$ satisfies the recursion

\begin{align*}
a_0(u) &= \delta(\textrm{sign}(u)-1)\\
a_k(u) &= a_{k-1}(u) \sum_{h_k\geq |u|}q_k(h_k) + a_{k-1}(-u) \sum_{h_k \leq -|h|} q_k(h_k)\\
\end{align*}

Then we finally have

$$p_k(u) = \cases{(a_k(u)-a_k(u+\textrm{sign}(u))) & for $u\neq0$\\
           1-\prod_{j=1}^k(1-q_j(0)) & for $u=0$}$$


In [3]:
function iter_factor(Q, J, y=0)
    a = fill(0.0, -J-1:J+1)
    a[0:J] .= 1
    for q ∈ Q
        Σp, Σm = 0.0, 0.0
        for h=J:-1:1
            ap, am = a[h], a[-h]
            Σp += q[h]; Σm += q[-h]
            a[+h] = ap*Σp + am*Σm
            a[-h] = am*Σp + ap*Σm
        end
        a[0] *= 1-q[0]
    end
    
    p = fill(0.0, -J:J)
    for u = 1:J
        p[+u] = a[+u] - a[+u+1]
        p[-u] = a[-u] - a[-u-1]
    end
    p[0] = 1-a[0]
    p ./= sum(p)
end


iter_factor (generic function with 2 methods)

Comparison
--

In [4]:
J=10
y=0.1
Q=[(p=fill(0.0,-J:J); p[-J:J] .= rand(2J+1); p ./=sum(p)) for i=1:1]
[iter_factor(Q,J,y) iter_slow_factor(Q,J,y)]

21×2 Array{Float64,2}:
 0.05798     0.05798
 0.00980312  0.00980312
 0.00202876  0.00202876
 0.0466536   0.0466536
 0.0487358   0.0487358
 0.0816751   0.0816751
 0.00929546  0.00929546
 0.0537315   0.0537315
 0.088983    0.088983
 0.0426416   0.0426416
 0.0171973   0.0171973
 0.0751703   0.0751703
 0.0835248   0.0835248
 0.0936016   0.0936016
 0.0337495   0.0337495
 0.0715908   0.0715908
 0.0533916   0.0533916
 0.0876417   0.0876417
 0.0115408   0.0115408
 0.015884    0.015884
 0.0151798   0.0151798

The expression for $C_{ia}$ is

\begin{align*}
    C_{ia} &=\left|s + \sum_{b=1}^d u_b\right| - \sum_{b=1}^d|u_b|
\end{align*}
and

\begin{align*}
q(h) &= \sum_{u_1,\dots,u_d} \prod_{b=1,\dots,d} p_b(u_b) \delta\left(h-s-\sum_{b=1}^du_b\right)e^{y C_{ia}}\\
&= e^{y |h|}\sum_{u_1,\dots,u_d} \prod_{b=1,\dots,d} p_b(u_b)e^{-y|u_b|} \delta\left(h-s-\sum_{b=1}^du_b\right)\\
\end{align*}


To compute $q_d$, note that

\begin{align}
q(h) &= q_d(h)e^{y|h|}
\end{align}

where $q_d$ satisfies
\begin{align}
q_0(h) & = \delta(h-s)\\
q_d(h) & = \sum_{u} q_{d-1}(h-u) p_d(u)e^{-y|u|} 
\end{align}

In [5]:
function ⊛(p1, p2)
    q = fill(0.0,firstindex(p1)+firstindex(p2):lastindex(p1)+lastindex(p2))
    for u1 in eachindex(p1), u2 in eachindex(p2)
        q[u1+u2] += p1[u1]*p2[u2]
    end
    q
end

function iter_var(P, s, J, y=0)
    q = fill(1.0, s:s)
    for p ∈ P
        q = q ⊛ (p .* exp.(-y .* abs.(eachindex(p))))
    end
    q .*= exp.(y .* abs.(eachindex(q)))
    qnew = fill(0.0, -J:J)
    for h in eachindex(q)
        qnew[clamp(h,-J,J)] += q[h]
    end
    qnew ./= sum(qnew)
end

iter_var (generic function with 2 methods)

Comparison
--

In [6]:
J=13
y=1.0
s=1
P=[(p=fill(0.0, -J:J); p[-J:J] .= rand(2J+1); p ./=sum(p)) for i=1:3]
[iter_var(P,s,J,y) iter_slow_var(P,s,J,y)]

27×2 Array{Float64,2}:
 0.0862894    0.0862894
 0.00414388   0.00414388
 0.00360311   0.00360311
 0.00323086   0.00323086
 0.00278224   0.00278224
 0.00214639   0.00214639
 0.00162998   0.00162998
 0.00122897   0.00122897
 0.000979949  0.000979949
 0.00081929   0.00081929
 0.000589318  0.000589318
 0.000337985  0.000337985
 0.000143272  0.000143272
 ⋮            
 0.000301125  0.000301125
 0.000734774  0.000734774
 0.0014851    0.0014851
 0.00293419   0.00293419
 0.00478464   0.00478464
 0.00717932   0.00717932
 0.00936764   0.00936764
 0.0123115    0.0123115
 0.0155457    0.0155457
 0.0193924    0.0193924
 0.0248399    0.0248399
 0.793025     0.793025

In [7]:
using StatsBase, ProgressMeter

uni(J) = fill(1/(2J+1), -J:J)
residual(x) = (p=OffsetVector((x .* eachindex(x))[1:end], 0:lastindex(x)-1); p./=sum(p))

function moments(popP)
    M = 0.0; V = 0.0
    for t =1:length(popP)
        m1 = sum(i*popP[t][i] for i in eachindex(popP[t]))
        m2 = sum(i^2*popP[t][i] for i in eachindex(popP[t]))
        M += m1/length(popP); 
        V +=(m2-m1^2)/length(popP) #V=0 if solution RS, V>0 otherwise
    end
    M, V
end

function RSB(Λ, K; 
        J=10, 
        maxiter=100, 
        popsize=1000, 
        popP = [uni(J) for i=1:popsize], 
        popQ = [uni(J) for i=1:popsize], 
        y=0)
    Λ1 = residual(Λ)
    K1 = residual(K)
    wΛ1 = weights(Λ1)
    wK1 = weights(K1)

    @showprogress for t = 1:maxiter
        for i = eachindex(popQ)
            d = sample(eachindex(Λ1), wΛ1)
            k = sample(eachindex(K1), wK1)
            Q = rand(popQ, k)
            P = rand(popP, d)
            s = rand((-1,1))
            popQ[i] = iter_var(P, s, J, y)
            popP[i] = iter_factor(Q, J, y)
        end
    end
    popP, popQ
end

RSB (generic function with 1 method)

In [8]:
Λ = OffsetVector([0,0,0.5,0.5], 0:3)
K = OffsetVector([0,0,0,1], 0:3)


J=10
popsize=10000
popQ = [OffsetVector(ones(2J+1)/(2J+1),-J:J) for i=1:popsize];
popP = [OffsetVector(ones(2J+1)/(2J+1),-J:J) for i=1:popsize];

In [9]:
popP, popQ = RSB(Λ,K; J=J, maxiter=100, popsize=popsize, popQ=popQ, popP=popP, y=5.0);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


In [10]:
using StatsBase

proportions(argmax.(popP))

21-element Array{Float64,1}:
 0.44570000000000004
 0.059300000000000005
 0.0035
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0026000000000000003
 0.058600000000000006
 0.4303

In [11]:
M, V = moments(popP)

(-0.14406464972997673, 16.772719323219082)

1RSB equations for Max-Sum (Survey Propagation) at finite parameter $y$
---
We consider the auxiliary statistical physics model
\begin{align}
    \mathbb{P}_y(\{u_{ai},h_{ia}\})=\frac{1}{\Theta(y)}\mathbb{I}\left(\{u_{ai},h_{ia}\}\text{sat Max-Sum}\right)e^{-yF(\{u_{ai},h_{ia}\})}
\end{align}
in which the variables are the Max-Sum messages living on the edges, and the constraints (that can be put in a factorized form) enforces the Max-Sum equations to be satisfied. Each Max-Sum solution represents a cluster. The Max-Sum solutions are weighted according to their Bethe free energy, which corresponds to the minimal energy (inside the cluster). In the $y\to\infty$ limit one keeps only the clusters with minimal energy

* The distributions $Q_{ia}(h_{ia})$, $P_{ai}(u_{ai})$ over Max-Sum messages obey the SP(y) equations :
\begin{align}
    Q_{ia}(h_{ia})&=\frac{1}{Z_{ia}}\sum_{\{u_{bi}\}_{b\in\partial i\setminus a}}\delta(h_{ia}-f^{MS}_{ia}(\{u_{bi}\}_{b\in\partial i\setminus a}; s_i))e^{-yC_{ia}}\prod_{b\in\partial i\setminus a}P_{bi}(u_{bi}) \\
    P_{ai}(u_{ai})&=\frac{1}{Z_{ai}}\sum_{\{h_{ja}\}_{j\in\partial a\setminus i}}\delta(u_{ai}-f^{MS}_{ai}(\{h_{ja}\}_{j\in\partial a\setminus i}))e^{-yC_{ai}}\prod_{j\in\partial a\setminus i}Q_{ja}(h_{ja})    
\end{align}
where $h_{ia}-f^{MS}_{ia}(\{u_{bi}\}_{b\in\partial i\setminus a}; s_i)$, and $u_{ai}-f^{MS}_{ai}(\{h_{ja}\}_{j\in\partial a\setminus i})$ are shorthand notation for the Max-Sum equations
\begin{align}
    h_{ia}(\sigma_i)&=s_i\sigma_i+ \sum_{b\in\partial i\setminus a}u_{bi}(\sigma_i) - C_{ia}\\
    u_{ai}(\sigma_i)&=\max_{\{\sigma_j\}_{j\in\partial a\setminus i}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}h_{ja}(\sigma_j)\right) - C_{ai}
\end{align}
and $C_{ia}$, $C_{ai}$ are the constants in the Max-Sum equations
\begin{align}
    C_{ia}&=\max_{\sigma_i}\left(s_i\sigma_i+ \sum_{b\in\partial i\setminus a}u_{bi}(\sigma_i)\right)\\
    C_{ai}&=\max_{\{\sigma_i\}_{i\in\partial a}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}h_{ja}(\sigma_j)\right)
\end{align}
the constants $Z_{ia}$, $Z_{ai}$ are normalizations of the distributions $Q_{ia}$, $Q_{ai}$ over the Max-Sum messages

* With the parametrization $u_{ai}(\sigma_i) = u_{ai}\sigma_i+u_{0,ai}$ and $h_{ia}(\sigma_i) = h_{ia}\sigma_i + h_{0,ia}$ we get :
\begin{equation}
    u_{0,ai} = -\frac{1}{\beta}\log(2{\rm cosh}(\beta u_{ai})\to-|u_{ai}|
\end{equation}
(and the same for $h_{0,ia}$). Therefore :
\begin{align*}
    C_{ia}&=\max_{\sigma_i}\left(\sigma_i\left(s_i+ \sum_{b\in\partial i\setminus a}u_{bi}\right)\right) + \sum_{b\in\partial i\setminus a}u_{0,bi}\\
    &=\left|s_i+ \sum_{b\in\partial i\setminus a}u_{bi}\right| - \sum_{b\in\partial i\setminus a}|u_{bi}|\\
&=|h_{ia}|- \sum_{b\in\partial i\setminus a}|u_{bi}|\\
\end{align*}

and
\begin{align*}
    C_{ai}&=\max_{\{\sigma_i\}_{i\in\partial a}:{\rm sat}}\left(\sum_{j\in\partial a\setminus i}h_{ja}\sigma_j\right)-\sum_{j\in\partial a\setminus i}|h_{ja}|\\
% &= -2\min_{j\in\partial a\setminus i}|h_{ja}|\Theta\left(-\prod_{j\in\partial a\setminus j}h_{ja}\right)
&=0
\end{align*}

* We can then write the same 1RSB equations for the distributions $Q_{ia}(h_{ia})$, $P_{ai}(u_{ai})$ over the integer parameters $h_{ia}$, $u_{ai}$:
\begin{align}
    P(u)&=\frac{1}{Z_P(y)}\sum_{h_1,\dots h_k}\delta\left(u-\min_{j=1\dots,k}|h_j|\prod_j{\rm sgn}(h_j)\right)e^{-yC_{ai}(h_1,\dots,h_k)}\prod_j Q_j(h_j) \\
    Q(h)&=\frac{1}{Z_Q(y)}\sum_{u_1,\dots u_d}\delta(h-s-\sum_{b=1}^du_b)e^{-yC_{ia}(u_1,\dots,u_d)}\prod_{b=1}^dP_b(u_b)
\end{align}

* Random graph ensemble
When the source and the factor graph are random variables the 1RSB messages $P_{ai}$, $(a,i)\in E$ become random variables. Let $(a,i)$ be a uniformly chosen edge in a factor graph drawn from the random ensembl, and let $\mathcal{P}$ be the distribution of the message $P_{ai}$ solution of the 1RSB equation written above. The distribution $\mathcal{P}(p)$ obey consistency equation similar to the RS cavity equations:
\begin{align}
    \mathcal{P}^{1RSB}(P)&=\sum_{k}\tilde{P}_k \int\prod_{j=1}^k{\rm d}\mathcal{Q}^{1RSB}(Q_j)\delta(P-F^{SP(y)}(Q_1,\dots,Q_k)) \\
    \mathcal{Q}^{1RSB}(Q) &= \sum_{s}\frac{1}{2}\sum_{d}\tilde{\Lambda_d}\int\prod_{b=1}^d{\rm d}\mathcal{P}^{1RSB}(P_b)\delta(Q-G^{SP(y)}(P_1,\dots,P_d;s))
\end{align}
This equation always admits a trivial fixed point $\mathcal{P}(P)=\sum_u p^{RS}(u)\delta[P-\delta(\cdot-u)]$, $\mathcal{Q}(h)=\sum_h q^{RS}(h)\delta[Q-\delta(\cdot-h)]$, where $p^{RS}, q^{RS}$  is the solution of the RS cavity equation.
In the RS phase, this trivial fixed-point is the unique solution, while in the 1RSB phase, the trivial solution becomes unstable and the above equation admits a non-trivial solution.

Average minimal energy : Free energy of the auxiliary model
---
We want to estimate the Free energy $\mathcal{F}(y)=-\frac{1}{yn}\log\Theta(y)$ of the auxiliary problem defined by the partition function $\Theta(y)$ on a single instance. Then we can average over the random ensemble of instances. In the large $y\to\infty$ limit, we obtain the averaged minimal energy.

On a single instance, once a fixed point of the SP(y) equations has been found on a single instance, one can compute the Bethe free energy of the auxiliary problem:
\begin{align}
    \mathcal{F}(y) = \frac{1}{n}\sum_{a}\mathcal{F}_a(\{Q_{ia}\}_{i\in\partial a};y) + \frac{1}{n}\sum_i\mathcal{F}_i(\{P_{ai}\}_{a\in\partial i};y) - \frac{1}{n}\sum_{(ia)}\mathcal{F}_{(ia)}(P_{ai},Q_{ia};y)
\end{align}
with:
\begin{align}
    \mathcal{F}_a(\{Q_{ia}\}_{i\in\partial a};y) &= -\frac{1}{y}\log\left(\sum_{\{h_{ia}\}_{i\in\partial a}}e^{-yF_a(\{h_{ia}\}_{i\in\partial a})}\prod_{i\in\partial a}Q_{ia}(h_{ia})\right)\\
    \mathcal{F}_i(\{P_{ai}\}_{a\in\partial i};y) &= -\frac{1}{y}\log\left(\sum_{\{u_{ai}\}_{a\in\partial i}}e^{-yF_i(\{u_{ai}\}_{a\in\partial i})}\prod_{a\in\partial i}P_{ai}(u_{ai})\right)\\
    \mathcal{F}_{(ia)}(P_{ai},Q_{ia};y) &= -\frac{1}{y}\log\left(\sum_{u_{ia}, h_{ia}}e^{-yF_{ia}(u_{ia}, h_{ia})}P_{ai}(u_{ai})Q_{ia}(h_{ia})\right)
\end{align}
where $F_a,F_i, F_{ia}$ are the factor, variable and edge terms of the Bethe free energy associated to one fixed point of the Max-Sum equation: 
\begin{align}
    F_a(\{h_{ia}\}_{i\in\partial a}) &= 2\min_{i\in\partial a}|h_{ia}|\Theta(-\prod_i h_{ia})\\
    F_i(\{u_{ai}\}_{a\in\partial i}) &= -|s_i + \sum_{a\in\partial i}u_{ai}| + \sum_a |u_{ai}|\\
    F_{(ia)}(u_{ai},h_{ia}) &= -|u_{ai} + h_{ia}| + |u_{ai}| + |h_{ia}| 
\end{align}
Averaging over the random ensemble of instances:
\begin{align}
    \mathcal{F}^{1RSB}(y) &= \alpha\sum_{k}P_k\int\prod_{i=1}^k{\rm d}\mathcal{Q}^{1RSB}(Q_i)\mathcal{F}_a(Q_1,\dots,Q_k;y) + \sum_d\Lambda_d\sum_s\frac{1}{2}\int\prod_{i=1}^d\mathcal{P}^{1RSB}(P_i)\mathcal{F}_i(P_1,\dots,P_d;s,y) \\
    &- \Lambda'(1)\int\mathcal{F}_{(ia)}\mathcal{P}^{1RSB}(P)\mathcal{Q}^{1RSB}(Q)(P,Q;y)
\end{align}


In [12]:
function overlap_slow_factor(Q, J, y=0.0)
    w = 0.0
    k = length(Q)
    for hs in Iterators.product(fill(-J:J,k)...)
        Fa = 2*minimum(abs.(hs))*(∏(hs) < 0)
        w += ∏(q[h] for (q,h) ∈ zip(Q,hs)) * exp(-y*Fa)
    end
    log(w)
end

function overlap_slow_var(P, s, J, y=0.0)
    w = 0.0
    d = length(P)
    for us in Iterators.product(fill(-J:J,d)...)
        h = sum(us) + s
        Fi = -abs(h) + sum(abs.(us)) 
        w += ∏(p[u] for (p,u) ∈ zip(P,us)) * exp(-y*Fi)
    end
    log(w)
end

function overlap_slow_edge(p, q, J, y=0.0)
    w = 0.0
    for h in -J:J
        for u in -J:J
            Fia = abs(u)+abs(h)-abs(u+h)
            w += p[u]*q[h]*exp(-y*Fia)
        end
    end
    log(w)
end

overlap_slow_edge (generic function with 2 methods)

In [13]:
Q = rand(popQ, 3);

In [14]:
overlap_slow_factor(Q, J, y)

-1.8562532113402433e-7

In [152]:
overlap_fast_factor(Q, J, y)

0.19759305944105465

In [15]:
function overlap1RSB(Λ, K; 
        popP, 
        popQ, 
        samples=length(popP), 
        y=0.0)

    J = lastindex(popQ[1])
    wΛ = weights(Λ)
    wK = weights(K)

    O_factor = 0.0
    O_var = 0.0
    O_edge = 0.0
    
    @showprogress for t = 1:samples
        k = sample(eachindex(K), wK)
        Q = rand(popQ, k)
        O_factor += overlap_slow_factor(Q, J, y)/samples

        d = sample(eachindex(Λ), wΛ)
        P = rand(popP, d)
        s = rand((-1,1))
        O_var += overlap_slow_var(P, s, J, y)/samples

        p = rand(popP)
        q = rand(popQ)
        O_edge += overlap_slow_edge(p, q, J, y)/samples
    end
    mK = sum(k*K[k] for k=eachindex(K))
    mΛ = sum(d*Λ[d] for d=eachindex(Λ))
    α = mΛ/mK
    1/y*(α*O_factor + O_var - mΛ*O_edge)
end

overlap1RSB (generic function with 1 method)

In [16]:
Λ = OffsetVector([0,0,0.5,0.5], 0:3)
K = OffsetVector([0,0,0,1], 0:3)
y = 1.0
popP, popQ = RSB(Λ,K; J=10, maxiter=10^4, popsize=popsize, y=y);

Progress: 100%|█████████████████████████████████████████| Time: 0:07:17


In [17]:
O = overlap1RSB(Λ,K; popP=popP, popQ=popQ, y=y, samples=10^4)

Progress: 100%|█████████████████████████████████████████| Time: 0:02:46


0.47887845767874393

In [18]:
D=(1-O)/2

0.26056077116062804

Simplifications of the terms in the 1RSB Bethe free energy $\mathcal{F}(y)$
---
We can simplify the 1st term
\begin{align}
    \mathcal{F}_a(Q_1,\dots,Q_k;y) &= -\frac{1}{y}\log\left(\mathcal{Z}_a\right)\\
    \mathcal{Z}_a &= \sum_{g_1,\dots,g_k}e^{-2y\min_{i=1,\dots, k}|g_i|\Theta(-\prod_i g_i)}\prod_{i=1}^kQ_i(g_i)
\end{align}
we get:
\begin{align}
    \mathcal{Z}_a &=  \sum_{g_1,\dots,g_k}1-\Theta(-\prod_i g_i) + \Theta(-\prod_i g_i)e^{-2y\min_i|g_i|}\prod_{i=1}^k Q_i(g_i)\\
    &= 1+\sum_{g_1,\dots,g_k}\Theta(-\prod_i g_i)\left(e^{-2y\min_i|g_i|}-1\right)\prod_{i=1}^kQ_i(g_i) \\
    &= 1+ \sum_{f=0}^{\infty}\left(e^{-2yf}-1\right)\sum_{g_1,\dots,g_k}\Theta(-\prod_i g_i)\delta(f-\min_i|g_i|)\prod_{i=1}^kQ_i(g_i)\\
    &= 1+\sum_{f=0}^{\infty}\left(e^{-2yf}-1\right)(a_k(-f)-a_k(-f-1))
\end{align}



In [19]:
function overlap_factor(Q, J, y=1.0)
    a = fill(0.0, -J-1:J+1)
    a[1:J] .= 1
    for q ∈ Q
        Σp, Σm = 0.0, 0.0
        for f=J:-1:1
            ap, am = a[f], a[-f]
            Σp += q[f]; Σm += q[-f]
            #Σp, Σm = ∑(q[f:N]), ∑(q[-N:-f])
            a[+f] = ap*Σp + am*Σm
            a[-f] = am*Σp + ap*Σm
        end
    end

    w=1+sum((a[-f]-a[-f-1])*(exp(-2*y*f)-1) for f in 0:J)
    log(w)
end


overlap_factor (generic function with 2 methods)

Comparison
---

In [20]:
J=6
y=1.0
Q=[(p=fill(0.0,-J:J); p[-J:J] .= rand(2J+1); p ./=sum(p)) for i=1:2]
[overlap_factor(Q,J,y) overlap_slow_factor(Q,J,y)]

1×2 Array{Float64,2}:
 -0.543381  -0.543381

We can also simplify the second term :
\begin{align}
    \mathcal{F}_i(P_1,\dots,P_d;s,y) &= -\frac{1}{y}\log\left(\mathcal{Z}_i\right)\\
    \mathcal{Z}_i &= \sum_{f_1,\dots,f_d}e^{-y(\sum_{a=1}^d |f_a|-|s + \sum_{a=1}^df_a|)}\prod_{a=1}^d P_a(f_a)
\end{align}
we get
\begin{align}
    \mathcal{Z}_i &= \sum_g e^{y|g|}\sum_{f_1,\dots,f_d}\delta(g-s-\sum_a f_a)\prod_{a=1}^d P_a(f_a)e^{-y |f_a|} \\
    &= \sum_g e^{y|g|}Q_d(g)
\end{align}
with
\begin{align}
    Q_0(g) &= \delta(g-s) \\
    Q_d(g) &= \sum_{f_d}Q_{d-1}(g-f_d)P_d(f_d)e^{-y|f_d|}
\end{align}


In [21]:
function overlap_var(P, s, J, y=0)
    q = fill(1.0, s:s)
    for p ∈ P
        q = q ⊛ (p .* exp.(-y .* abs.(eachindex(p))))
    end
    w = sum(q[g]*exp(y*abs(g)) for g in eachindex(q))
    log(w)
end

overlap_var (generic function with 2 methods)

Comparison
---

In [22]:
J=13
y=1.0
s=1
P=[(p=fill(0.0, -J:J); p[-J:J] .= rand(2J+1); p ./=sum(p)) for i=1:3]
[overlap_var(P,s,J,y) overlap_slow_var(P,s,J,y)]

1×2 Array{Float64,2}:
 -0.986327  -0.986327